# Financial Behavior Analysis

## Exploratory Data Analysis(ُEDA)

**Importing  libraries**

In [ ]:
%pip install ydata-profiling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
from ydata_profiling import ProfileReport
from scipy import stats

**Load & Explore Data**

In [ ]:
df = pd.read_csv('financial_behavior_dataset.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

**Clean Data**

In [ ]:
# Convert Month to datetime
df["Month"] = pd.to_datetime(df["Month"], format="%m-%Y", errors="coerce").dt.month

In [ ]:
#Data Type Check
print(df.dtypes)

In [ ]:
df.isnull().sum()

In [ ]:
#replaces null cell with 0
df['Savings_Amount'] = df['Savings_Amount'].fillna(0)

In [ ]:
df['Savings_Amount'].isnull().sum()

In [ ]:
# Duplicate rows
df.duplicated().sum()

**Descriptive Statistics**

In [ ]:
df.describe()

In [ ]:
# Analysis of numerical variables (such as spending and saving)
num_cols = ['Age', 'Income', 'Total_Spending', 'Essential_Spending',
            'Non_Essential_Spending', 'Savings_Amount',
            'Investments_Amount', 'Debt_Payments', 'Account_Balance']

df[num_cols].hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()


In [ ]:
#Relationships between Age,income,spending, saving, and debt
sns.pairplot(df[num_cols])
plt.show()

In [ ]:
#Distribution by gender
sns.boxplot(x='Gender', y='Total_Spending', data=df)

In [ ]:
# Distribution by month
sns.barplot(x='Month', y='Savings_Amount', data=df)

In [ ]:
# Check the correlation
plt.figure(figsize=(10, 6))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

**Derivation of indicators**

In [ ]:
#Financial Volatility Index
# If Spending_Volatility is high → the client is "seasonal" or "random"
Spending_Volatility = df.groupby("Customer_ID")["Total_Spending"].std().reset_index()
Spending_Volatility.columns = ["Customer_ID", "Spending_Volatility"]
Spending_Volatility

In [ ]:
# Savings Rate
df["Savings_Rate"] = df["Savings_Amount"] / df["Income"]
df["Savings_Rate"]

In [ ]:
# Investment Rate
df["Investment_Rate"] = df["Investments_Amount"] / df["Income"]
df["Investment_Rate"]

In [ ]:
# Debt to Income Ratio
df["Debt_Ratio"] = df["Debt_Payments"] / df["Income"]
df["Debt_Ratio"]

In [ ]:
# Total Spending Rate
df["Spending_Rate"] = df["Total_Spending"] / df["Income"]
df["Spending_Rate"]

In [ ]:
# Merge the Spending_Volatility DataFrame with the main DataFrame only if Spending_Volatility column doesn't exist
if "Spending_Volatility" not in df.columns:
  df = pd.merge(df, Spending_Volatility, on="Customer_ID", how="left")

# Calculate initial customer metrics
customer_metrics = df.groupby("Customer_ID")[["Savings_Rate", "Investment_Rate", "Debt_Ratio", "Spending_Volatility", "Spending_Rate", "Total_Spending"]].mean().reset_index()
display(customer_metrics)

**Financial Behavior Type and Financial Goal Type**

In [ ]:

def classify_behavior(row):
    if row["Savings_Rate"] > 0.1 and row["Investment_Rate"] > 0.1:
        return "disciplined"
    elif row["Spending_Volatility"] > 2000:
        return "volatile"
    elif row["Savings_Rate"] < 0.05 and row["Investment_Rate"] < 0.05:
        return "spender"
    else:
        return "unclassified"
customer_metrics["Behavior_Type"] = customer_metrics.apply(classify_behavior, axis=1)


In [ ]:
def classify_goal(row):
    if row["Savings_Rate"] > 0.10 and row["Investment_Rate"] < 0.05:
        return "Saver"
    elif row["Investment_Rate"] > 0.1:
        return "Investor"
    else:
        return "Spender"
customer_metrics["Financial_Goal"] = customer_metrics.apply(classify_goal, axis=1)

In [ ]:
display(customer_metrics)

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='Behavior_Type', data=customer_metrics, order=customer_metrics["Behavior_Type"].value_counts().index)
plt.title("Client Distribution by Financial Behavior")
plt.xlabel("Financial behavior ")
plt.ylabel("Number of clients")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
customer_metrics["Behavior_Type"].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title("The percentage of each type of financial behavior")
plt.ylabel("")  # Hide sidebar
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='Financial_Goal', data=customer_metrics, order=customer_metrics["Financial_Goal"].value_counts().index)
plt.title("Number of clients by financial goal")
plt.xlabel("financial goal")
plt.ylabel("Number of clients")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
customer_metrics["Financial_Goal"].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title("The percentage of each financial goal")
plt.ylabel("")  # Hide sidebar
plt.show()

In [ ]:
#Average spending by behavior
plt.figure(figsize=(10, 6))
sns.barplot(x=customer_metrics['Behavior_Type'], y=df['Total_Spending'], data=df, estimator='mean')
plt.title("Average spending by financial behavior")
plt.xlabel("Financial behavior")
plt.ylabel("Average spending")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#Comparing behavior with financial goals
plt.figure(figsize=(10, 6))
sns.countplot(x="Behavior_Type", hue='Financial_Goal', data=customer_metrics, order=customer_metrics["Behavior_Type"].value_counts().index)
plt.title("Distributing financial goals according to behavioral")
plt.xlabel("financial behavior")
plt.ylabel("Number of clients")
plt.xticks(rotation=45)
plt.legend(title='financial goals')
plt.tight_layout()
plt.show()

In [ ]:
df = df.drop(columns=['Spending_Volatility_y', 'Spending_Volatility_x'])

**Save the customer classification to csv**

In [ ]:
# save the customer classification to CSV file
customer_metrics.to_csv("customer_classification_data.csv", index=False)

**Generate report**

In [ ]:
# Generate a quick, comprehensive automated report
profile = ProfileReport(df, title="EDA Report", explorative=True)
profile.to_file("eda_report.html")